# REDUCEDHEATCARB sanity check

In [ ]:
import pandas as pd
import numpy as np

rhc_file_path='reducedheatcarb_raw_measurements.parquet'
# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

import sys
sys.path.append('../data/')
sys.path.append('../view/')

%load_ext autoreload


%matplotlib inline
%matplotlib widget
import pylab as plt
import itertools
from plotter import Plot
from tqdm.notebook import tqdm
from preprocessor import Preprocessor

In [ ]:
units_to_mathtext = property_types = {
    'ppm' : r'$ppm$',
    'kWh' : r'$kWh$',
    'm3' : r'$m^{3}$',
    'degC' : r'$°C$',
    'W' : r'$W$',
    'V' : r'$V$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'W_m_2' : r'$W\cdotm^{-1}$'
}

In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df = pd.read_parquet(
        rhc_file_path, 
        engine='pyarrow',
        use_nullable_dtypes=True
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")


In [ ]:
df.info()

In [ ]:
df

## Initial exploration: size, ids, start & stop times per id

In [ ]:
df.info()

In [ ]:
# As it turns out, the original reducedheatcarb dataset (without Enelogic data added afterwards) does not contain duplicates
# As the code below typiclaly takes 1 - 2 minutes to run on a powerful serer, the code below which is commented out
# you only need to deduplicate after adding data, e.g. after downloading more Enelogic data

# %%time
# # deduplicate the measurements
# df = df.reset_index().drop_duplicates().set_index(['id', 'source_category', 'source_type', 'timestamp', 'property']).sort_index()

In [ ]:
%%time
df_minmaxpersource = df.reset_index().groupby(['id', 'source_type'])['timestamp'].agg(['min', 'max'])

In [ ]:
df_minmaxpersource

In [ ]:
df.describe()

In [ ]:
print('#ids: ', len(df.index.get_level_values('id').unique()))
print('ids: ', list(df.index.get_level_values('id').unique()))

print('source categories: ', list(df.index.get_level_values('source_category').unique()))
print('source types: ', list(df.index.get_level_values('source_type').unique()))
print('properties: ', list(df.index.get_level_values('property').unique()))

print('values: ', df['value'].count())
print('#values per: ', df.groupby(['source_type']).size())
print('#values per: ', df.groupby(['source_category', 'source_type', 'property']).size())

## Remove ids with only batch_import data from Remeha

In [ ]:
df_size_per_id_and_source = df.groupby(['id', 'source_category']).size().unstack()


# Select ids where both 'cloud_feed' and 'device' are not greater than 0
filtered_ids = df_size_per_id_and_source[(df_size_per_id_and_source['cloud_feed'].fillna(0) <= 0) & (df_size_per_id_and_source['device'].fillna(0) <= 0)].index.tolist()

print("Removing ids with only Remeha data, i.e. where both 'cloud_feed' and 'device' are not > 0:")
print(filtered_ids)

# Remove rows associated with ids in filtered_ids
df = df[~df.index.get_level_values('id').isin(filtered_ids)]

In [ ]:
print('#ids: ', len(df.index.get_level_values('id').unique()))
print('ids: ', list(df.index.get_level_values('id').unique()))

print('source categories: ', list(df.index.get_level_values('source_category').unique()))
print('source types: ', list(df.index.get_level_values('source_type').unique()))
print('properties: ', list(df.index.get_level_values('property').unique()))

print('values: ', df['value'].count())
print('#values per: \n', df.groupby(['source_type']).size())
print('#values per: \n', df.groupby(['source_category', 'source_type', 'property']).size())

In [ ]:
df.info()

In [ ]:
df.value.count()

## Add unit to hearbeat property

In [ ]:
%%time

# Boolean indexing to select rows where property equals 'heartbeat'
heartbeat_rows = df.index.get_level_values('property') == 'heartbeat'

# Reset index to modify column names
df.reset_index(inplace=True)

# Rename the 'property' column to 'heartbeat__0'
df.loc[heartbeat_rows, 'property'] = 'heartbeat__0'

# Set the index again
df.set_index(['id', 'source_category', 'source_type', 'timestamp', 'property'], inplace=True)

In [ ]:
print('#ids: ', len(df.index.get_level_values('id').unique()))
print('ids: ', list(df.index.get_level_values('id').unique()))

print('source categories: ', list(df.index.get_level_values('source_category').unique()))
print('source types: ', list(df.index.get_level_values('source_type').unique()))
print('properties: ', list(df.index.get_level_values('property').unique()))

print('values: ', df['value'].count())
print('#values per: \n', df.groupby(['source_type']).size())
print('#values per: \n', df.groupby(['source_category', 'source_type', 'property']).size())

## Preprocessing categorical data

In [ ]:
%%time
# Extract the 'boiler_status__str' values from the DataFrame
print("Unique categories for 'boiler_status__str':", list(df[df.index.get_level_values('property') == 'boiler_status__str']['value'].unique()))



In [ ]:
# Rename columns based on the translation table
boiler_status_categories = {
    '0.0': 'boiler_status_standby',
    '1.0': 'boiler_status_heat_demand',
    '2.0': 'boiler_status_burner_start',
    '3.0': 'boiler_status_burning_ch',
    '4.0': 'boiler_status_burning_dhw',
    '5.0': 'boiler_status_burner_stop',
    '6.0': 'boiler_status_pump_post_run',
    '7.0': 'boiler_status_cooling_active',
    '8.0': 'boiler_status_controlled_stop',
    '9.0': 'boiler_status_blocking_mode',
    '10.0': 'boiler_status_locking_mode',
    '11.0': 'boiler_status_cs_mode_l_ch',
    '12.0': 'boiler_status_cs_mode_h_ch',
    '13.0': 'boiler_status_cs_mode_hdhw',
    '15.0': 'boiler_status_manual_hd_ch_on',
    '16.0': 'boiler_status_boiler_frost_prot',
    '17.0': 'boiler_status_de_air',
    '18.0': 'boiler_status_cu_cooling',
    '19.0': 'boiler_status_reset_in_progress',
    '20.0': 'boiler_status_auto_filling',
    '21.0': 'boiler_status_halted',
    '22.0': 'boiler_status_forced_calibration',
    '23.0': 'boiler_status_factory_test',
    '200.0': 'boiler_status_device_mode',
    '254.0': 'boiler_status_unknown'
}

In [ ]:
%%time
df = Preprocessor.encode_categorical_property_as_boolean_properties(df, 'boiler_status__str', boiler_status_categories)

In [ ]:
print('#ids: ', len(df.index.get_level_values('id').unique()))
print('ids: ', list(df.index.get_level_values('id').unique()))

print('source categories: ', list(df.index.get_level_values('source_category').unique()))
print('source types: ', list(df.index.get_level_values('source_type').unique()))
print('properties: ', list(df.index.get_level_values('property').unique()))

print('values: ', df['value'].count())
print('#values per: \n', df.groupby(['source_type']).size())
print('#values per: \n', df.groupby(['source_category', 'source_type', 'property']).size())

In [ ]:
# Extract the 'gas_valve__str' values from the DataFrame
print('unique gas_valve__str values: ', list(df[df.index.get_level_values('property') == 'gas_valve__str']['value'].unique()))


In [ ]:
# Rename columns based on the translation table
gas_valve_categories = {
    '0.0': 'gas_valve_open',
    '1.0': 'gas_valve_closed',
    '2.0': 'gas_valve_off'
}


In [ ]:
%%time
df = Preprocessor.encode_categorical_property_as_boolean_properties(df, 'gas_valve__str', gas_valve_categories)

In [ ]:
print('#ids: ', len(df.index.get_level_values('id').unique()))
print('ids: ', list(df.index.get_level_values('id').unique()))

print('source categories: ', list(df.index.get_level_values('source_category').unique()))
print('source types: ', list(df.index.get_level_values('source_type').unique()))
print('properties: ', list(df.index.get_level_values('property').unique()))

print('values: ', df['value'].count())
print('#values per: \n', df.groupby(['source_type']).size())
print('#values per: \n', df.groupby(['source_category', 'source_type', 'property']).size())

## Get unique combinations per id for smart meter type and version

In [ ]:
# Extract the 'meter_code__str' values from the DataFrame
print('unique meter_code__str values: ', list(df[df.index.get_level_values('property') == 'meter_code__str']['value'].unique()))

# Extract the 'dsmr_version__0' values from the DataFrame
print('unique dsmr_version__0 values: ', list(df[df.index.get_level_values('property') == 'dsmr_version__0']['value'].unique()))

# Filter rows with 'meter_code__str' or 'dsmr_version__0' property
filtered_df = df[df.index.get_level_values('property').isin(['meter_code__str', 'dsmr_version__0'])]

# Unstack the DataFrame
unstacked_df = filtered_df.unstack(level='property')

# Find unique combinations for each id
unique_combinations = unstacked_df.groupby(level='id').agg(lambda x: tuple(set(x.dropna())))

# Display unique combinations for each id
print(unique_combinations)

## Remove smart meter type and version

In [ ]:
%%time
# Filter rows with 'meter_code__str' or 'dsmr_version__0' property
df = df[~df.index.get_level_values('property').isin(['meter_code__str', 'dsmr_version__0'])]

In [ ]:
print('#ids: ', len(df.index.get_level_values('id').unique()))
print('ids: ', list(df.index.get_level_values('id').unique()))

print('source categories: ', list(df.index.get_level_values('source_category').unique()))
print('source types: ', list(df.index.get_level_values('source_type').unique()))
print('properties: ', list(df.index.get_level_values('property').unique()))

print('values: ', df['value'].count())
print('#values per: \n', df.groupby(['source_type']).size())
print('#values per: \n', df.groupby(['source_category', 'source_type', 'property']).size())

In [ ]:
df.info()

## Convert all values to floats

In [ ]:
# Create a copy of the DataFrame
df_copy = df.copy(deep=True)

# Convert values to numeric in the copied DataFrame
numeric_values = pd.to_numeric(df_copy['value'], errors='coerce').values

# Now assign the modified column back to the original DataFrame
df_copy['value'] = numeric_values

In [ ]:
df = df_copy

In [ ]:
df.info()

In [ ]:
df

## Plot data


In [ ]:
Plot.nfh_measurements_plot(df, properties = ['temp_in__degC'], ids = [412715])


In [ ]:
Plot.nfh_measurements_plot(df, properties = ['temp_in__degC'])

In [ ]:
Plot.nfh_measurements_plot(df, properties = ['battery_voltage__V', 'co2__ppm'])

In [ ]:
# Plot.nfh_measurements_plot(df, source_types = ['enelogic'], ids = [412715])
# Plot.nfh_measurements_plot(df, source_types = ['enelogic'], ids = [404873])

Plot.nfh_measurements_plot(df, source_types = ['enelogic'])


In [ ]:
# Plot.nfh_measurements_plot(df, source_types = ['twomes-p1-reader-firmware'], ids = [412715])
# Plot.nfh_measurements_plot(df, source_types = ['twomes-p1-reader-firmware'], ids = [404873])

Plot.nfh_measurements_plot(df, source_types = ['twomes-p1-reader-firmware'])


In [ ]:
Plot.nfh_measurements_plot(df, source_types = ['enelogic', 'twomes-p1-reader-firmware'], units = ['kWh'], ids = [412715])
# Plot.nfh_measurements_plot(df, source_types = ['enelogic', 'twomes-p1-reader-firmware'], ids = [404873])
# Plot.nfh_measurements_plot(df, source_types = ['enelogic', 'twomes-p1-reader-firmware'])

In [ ]:
Plot.nfh_measurements_plot(df, source_types = ['twomes-co2-occupancy-scd41-m5coreink-firmware'], ids = [412715])
# Plot.nfh_measurements_plot(df, source_types = ['twomes-co2-occupancy-scd41-m5coreink-firmware'], ids = [404873])
# Plot.nfh_measurements_plot(df, source_types = ['twomes-co2-occupancy-scd41-m5coreink-firmware'])

In [ ]:
Plot.nfh_measurements_plot(df, source_types = ['twomes-co2-occupancy-scd41-m5coreink-firmware'], properties=['co2__ppm', 'onboarded__p', 'occupancy__p', 'temp_in__degC'], ids = [456638])
# Plot.nfh_measurements_plot(df, source_types = ['twomes-co2-occupancy-scd41-m5coreink-firmware'], ids = [404873])
# Plot.nfh_measurements_plot(df, source_types = ['twomes-co2-occupancy-scd41-m5coreink-firmware'])

In [ ]:
Plot.nfh_measurements_plot(df, properties=['temp_in__degC'])

In [ ]:
Plot.nfh_measurements_plot(df, properties=['co2__ppm'])


In [ ]:
Plot.nfh_measurements_plot(df, ids = [412715])
# Plot.nfh_measurements_plot(df, ids = [404873])
# Plot.nfh_measurements_plot(df)